In [15]:
import pandas as pd
from datetime import datetime, timedelta

# Load the CSV file
file_path = "dataForTests.csv"
df = pd.read_csv(file_path)

# List of columns to keep
columns_to_keep = [
    "File Start Date", "File End Date", "File Start Time", "File End Time", 
    "File Duration", "Hectometer Head", "Hectometer Tail", "Route Letter", 
    "Route Number", "Route Description", "Hectometering Direction", 
    "Head Road Section From", "Head Road Section To", "Trajectory From", "Trajectory To"
]

# Drop all other columns except the ones listed above
df_filtered = df[columns_to_keep]

# Display the first few rows of the filtered dataframe
df_filtered.head()

/tmp/ipykernel_25930/148216218.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,File Start Date,File End Date,File Start Time,File End Time,File Duration,Hectometer Head,Hectometer Tail,Route Letter,Route Number,Route Description,Hectometering Direction,Head Road Section From,Head Road Section To,Trajectory From,Trajectory To
0,2022-04-04,2022-04-04,08:02:29,08:09:00,6.517,21.7,25.4,A,4,A4,descending,Roelofarendsveen,Ringvaart-Aquaduct,Den Haag,Amsterdam
1,2022-04-04,2022-04-04,08:02:29,08:10:02,7.550,26.8,29.3,A,4,A4,descending,Hoogmade,Roelofarendsveen,Den Haag,Amsterdam
2,2022-04-04,2022-04-04,08:03:30,08:06:00,2.500,53.0,56.2,A,16,A16,descending,Zonzeel,Zevenbergschen Hoek,Breda,Rotterdam
3,2022-04-04,2022-04-04,08:03:30,08:07:00,3.500,31.0,32.9,A,27,A27,descending,Werkendam,Nieuwendijk,Gorinchem,Breda
4,2022-04-04,2022-04-04,08:03:30,08:42:03,38.550,0.0,2.3,A,44,A44,descending,Nieuw-Vennep,Burgerveen,Wassenaar,Amsterdam


In [16]:
# Create a copy of the filtered DataFrame to avoid SettingWithCopyWarning
df_filtered_copy = df_filtered.copy()

# Converting time columns to datetime for easier filtering
df_filtered_copy['File Start Time'] = pd.to_datetime(df_filtered_copy['File Start Time'], format='%H:%M:%S').dt.time
df_filtered_copy['File End Time'] = pd.to_datetime(df_filtered_copy['File End Time'], format='%H:%M:%S').dt.time

# Adding a new column to check if the start hour is different from the end hour
df_filtered_copy['Different Start-End Hour'] = df_filtered_copy['File Start Time'].apply(lambda x: x.hour) != df_filtered_copy['File End Time'].apply(lambda x: x.hour)

pd.set_option('display.max_rows', 10)
df_filtered_copy.head(10)



,File Start Date,File End Date,File Start Time,File End Time,File Duration,Hectometer Head,Hectometer Tail,Route Letter,Route Number,Route Description,Hectometering Direction,Head Road Section From,Head Road Section To,Trajectory From,Trajectory To,Different Start-End Hour
0,2022-04-04,2022-04-04,08:02:29,08:09:00,6.517,21.7,25.4,A,4,A4,descending,Roelofarendsveen,Ringvaart-Aquaduct,Den Haag,Amsterdam,False
1,2022-04-04,2022-04-04,08:02:29,08:10:02,7.550,26.8,29.3,A,4,A4,descending,Hoogmade,Roelofarendsveen,Den Haag,Amsterdam,False
2,2022-04-04,2022-04-04,08:03:30,08:06:00,2.500,53.0,56.2,A,16,A16,descending,Zonzeel,Zevenbergschen Hoek,Breda,Rotterdam,False
3,2022-04-04,2022-04-04,08:03:30,08:07:00,3.500,31.0,32.9,A,27,A27,descending,Werkendam,Nieuwendijk,Gorinchem,Breda,False
4,2022-04-04,2022-04-04,08:03:30,08:42:03,38.550,0.0,2.3,A,44,A44,descending,Nieuw-Vennep,Burgerveen,Wassenaar,Amsterdam,False
5,2022-04-04,2022-04-04,08:20:30,08:25:01,4.517,41.5,44.2,A,2,A2,descending,Vinkeveen,Abcoude,Utrecht,Amsterdam,False
6,2022-04-04,2022-04-04,08:20:30,08:58:00,37.500,79.6,83.0,A,27,A27,descending,De Bilt,Rijnsweerd,Utrecht,Gorinchem,False
7,2022-04-04,2022-04-04,08:20:30,08:22:00,1.500,19.1,21.1,A,58,A58,descending,Oirschot,Best,Tilburg,Eindhoven,False
8,2022-04-04,2022-04-04,08:20:30,08:51:01,30.517,159.4,161.4,N,2,N2,descending,Eindhoven-Centrum,Eindhoven-Airport,Leenderheide,Batadorp,False
9,2022-04-04,2022-04-04,08:21:30,09:01:01,39.517,52.3,54.3,A,15,A15,descending,Rotterdam-Heijplaat,Benelux,Ridderkerk,Europoort,True


In [17]:
df_filtered_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   File Start Date           99999 non-null  object 
 1   File End Date             99999 non-null  object 
 2   File Start Time           99999 non-null  object 
 3   File End Time             99999 non-null  object 
 4   File Duration             99999 non-null  float64
 5   Hectometer Head           99999 non-null  float64
 6   Hectometer Tail           99999 non-null  float64
 7   Route Letter              99999 non-null  object 
 8   Route Number              99999 non-null  int64  
 9   Route Description         99999 non-null  object 
 10  Hectometering Direction   99999 non-null  object 
 11  Head Road Section From    99999 non-null  object 
 12  Head Road Section To      99999 non-null  object 
 13  Trajectory From           99999 non-null  object 
 14  Trajec

In [18]:
import pandas as pd
from datetime import datetime, timedelta

# Assuming df_filtered_copy is already defined and processed as in your previous code

# Function to round time to the next full hour
def round_to_next_hour(t):
    return (datetime.combine(datetime.today(), t) + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0).time()

# Splitting the rows
split_rows = []
for index, row in df_filtered_copy.iterrows():
    if row['Different Start-End Hour']:
        # Create a new row with the end time adjusted to the next full hour
        new_row_1 = row.copy()
        new_row_1['File End Time'] = round_to_next_hour(row['File Start Time'])
        split_rows.append(new_row_1)

        # Create another new row with the start time adjusted from the original end time
        new_row_2 = row.copy()
        new_row_2['File Start Time'] = round_to_next_hour(row['File End Time'])
        split_rows.append(new_row_2)

# Add the new rows to the DataFrame
for new_row in split_rows:
    df_filtered_copy = df_filtered_copy.append(new_row, ignore_index=True)

# Optional: Remove the original rows where start and end hour were different, if needed
df_filtered_copy = df_filtered_copy[~df_filtered_copy['Different Start-End Hour']]

# Reset the index if you have removed rows
df_filtered_copy.reset_index(drop=True, inplace=True)

# Display the DataFrame
pd.set_option('display.max_rows', 150)
df_filtered_copy.head(20)


/tmp/ipykernel_25930/3057473042.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_filtered_copy = df_filtered_copy.append(new_row, ignore_index=True)


   File Start Date File End Date File Start Time File End Time  File Duration  \
0       2022-04-04    2022-04-04        08:02:29      08:09:00          6.517   
1       2022-04-04    2022-04-04        08:02:29      08:10:02          7.550   
2       2022-04-04    2022-04-04        08:03:30      08:06:00          2.500   
3       2022-04-04    2022-04-04        08:03:30      08:07:00          3.500   
4       2022-04-04    2022-04-04        08:03:30      08:42:03         38.550   
5       2022-04-04    2022-04-04        08:20:30      08:25:01          4.517   
6       2022-04-04    2022-04-04        08:20:30      08:58:00         37.500   
7       2022-04-04    2022-04-04        08:20:30      08:22:00          1.500   
8       2022-04-04    2022-04-04        08:20:30      08:51:01         30.517   
9       2022-04-04    2022-04-04        08:39:32      08:43:00          3.467   
10      2022-04-04    2022-04-04        08:39:32      08:42:03          2.517   
11      2022-04-04    2022-0

In [19]:
# Adding a dummy date to the time fields
dummy_date = datetime(2000, 1, 1)  # The date doesn't matter, it's just a placeholder

# Convert time to string and concatenate with dummy date
df_filtered_copy['start_datetime_full'] = pd.to_datetime(
    dummy_date.strftime('%Y-%m-%d') + ' ' + df_filtered_copy['File Start Time'].apply(lambda t: t.strftime('%H:%M:%S'))
)
df_filtered_copy['end_datetime_full'] = pd.to_datetime(
    dummy_date.strftime('%Y-%m-%d') + ' ' + df_filtered_copy['File End Time'].apply(lambda t: t.strftime('%H:%M:%S'))
)

# Calculate the time difference
df_filtered_copy['time_difference'] = df_filtered_copy['end_datetime_full'] - df_filtered_copy['start_datetime_full']

# Convert time difference to a desired format, like total seconds
df_filtered_copy['time_difference_seconds'] = df_filtered_copy['time_difference'].dt.total_seconds()

# Display the DataFrame
pd.set_option('display.max_rows', 10)
df_filtered_copy[['start_datetime_full', 'end_datetime_full', 'time_difference', 'time_difference_seconds']].head(10)

,start_datetime_full,end_datetime_full,time_difference,time_difference_seconds
0,2000-01-01 08:02:29,2000-01-01 08:09:00,0 days 00:06:31,391.0
1,2000-01-01 08:02:29,2000-01-01 08:10:02,0 days 00:07:33,453.0
2,2000-01-01 08:03:30,2000-01-01 08:06:00,0 days 00:02:30,150.0
3,2000-01-01 08:03:30,2000-01-01 08:07:00,0 days 00:03:30,210.0
4,2000-01-01 08:03:30,2000-01-01 08:42:03,0 days 00:38:33,2313.0
5,2000-01-01 08:20:30,2000-01-01 08:25:01,0 days 00:04:31,271.0
6,2000-01-01 08:20:30,2000-01-01 08:58:00,0 days 00:37:30,2250.0
7,2000-01-01 08:20:30,2000-01-01 08:22:00,0 days 00:01:30,90.0
8,2000-01-01 08:20:30,2000-01-01 08:51:01,0 days 00:30:31,1831.0
9,2000-01-01 08:39:32,2000-01-01 08:43:00,0 days 00:03:28,208.0


In [20]:
# Function to safely convert timedelta to time, handling negative differences
def timedelta_to_time(td):
    if pd.isna(td) or td.total_seconds() < 0:
        return None  # Or handle negative time difference as needed
    return (datetime.min + td).time()

# Convert the timedelta to time format (hours, minutes, seconds), handling negative values
df_filtered_copy['time_difference_time'] = df_filtered_copy['time_difference'].apply(timedelta_to_time)


# Dropping the specified columns
df_filtered_copy.drop(['start_datetime_full', 'end_datetime_full', 'time_difference'], axis=1, inplace=True)

# Display the updated DataFrame
pd.set_option('display.max_rows', 10)
df_filtered_copy.head(10)

,File Start Date,File End Date,File Start Time,File End Time,File Duration,Hectometer Head,Hectometer Tail,Route Letter,Route Number,Route Description,Hectometering Direction,Head Road Section From,Head Road Section To,Trajectory From,Trajectory To,Different Start-End Hour,time_difference_seconds,time_difference_time
0,2022-04-04,2022-04-04,08:02:29,08:09:00,6.517,21.7,25.4,A,4,A4,descending,Roelofarendsveen,Ringvaart-Aquaduct,Den Haag,Amsterdam,False,391.0,00:06:31
1,2022-04-04,2022-04-04,08:02:29,08:10:02,7.550,26.8,29.3,A,4,A4,descending,Hoogmade,Roelofarendsveen,Den Haag,Amsterdam,False,453.0,00:07:33
2,2022-04-04,2022-04-04,08:03:30,08:06:00,2.500,53.0,56.2,A,16,A16,descending,Zonzeel,Zevenbergschen Hoek,Breda,Rotterdam,False,150.0,00:02:30
3,2022-04-04,2022-04-04,08:03:30,08:07:00,3.500,31.0,32.9,A,27,A27,descending,Werkendam,Nieuwendijk,Gorinchem,Breda,False,210.0,00:03:30
4,2022-04-04,2022-04-04,08:03:30,08:42:03,38.550,0.0,2.3,A,44,A44,descending,Nieuw-Vennep,Burgerveen,Wassenaar,Amsterdam,False,2313.0,00:38:33
5,2022-04-04,2022-04-04,08:20:30,08:25:01,4.517,41.5,44.2,A,2,A2,descending,Vinkeveen,Abcoude,Utrecht,Amsterdam,False,271.0,00:04:31
6,2022-04-04,2022-04-04,08:20:30,08:58:00,37.500,79.6,83.0,A,27,A27,descending,De Bilt,Rijnsweerd,Utrecht,Gorinchem,False,2250.0,00:37:30
7,2022-04-04,2022-04-04,08:20:30,08:22:00,1.500,19.1,21.1,A,58,A58,descending,Oirschot,Best,Tilburg,Eindhoven,False,90.0,00:01:30
8,2022-04-04,2022-04-04,08:20:30,08:51:01,30.517,159.4,161.4,N,2,N2,descending,Eindhoven-Centrum,Eindhoven-Airport,Leenderheide,Batadorp,False,1831.0,00:30:31
9,2022-04-04,2022-04-04,08:39:32,08:43:00,3.467,4.7,6.4,A,73,A73,descending,Maasbracht,Het Vonderen,Nijmegen,Maasbracht,False,208.0,00:03:28
